In [4]:
import os
import sys
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import lib.params as params
import lib.network as network
import lib.utils as utils
from datetime import datetime
import time

In [5]:
%load_ext autoreload
%autoreload 2
#read params
with open("config/train.params") as f:
    learning_rate=float(params.read_param(f.readline()))
    batch_size=int(params.read_param(f.readline()))
    epoch=int(params.read_param(f.readline()))
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))
    
data_all=np.load("out/all_data.npy")  
label_all=np.load("out/all_labels.npy")
# print(data_all.shape,label_all.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
training with a learning rate of 0.1 for 1 epochs with batchs of size 3


In [10]:
%matplotlib inline
print("init network")
net=network.R2N2(learning_rate)
model_dir="out/model_{}_{}_{} {}".format(learning_rate,epoch,batch_size,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

# train network
loss_all=[]
acc_all=[]
N=len(data_all)
print("training")
for e in range(epoch):
    start_time=time.time()
    perm=np.random.permutation(N)
    data_all=data_all[perm]
    label_all=label_all[perm]
    split_size=math.ceil(N/batch_size)
    data_batchs=np.array_split(data_all,split_size)
    label_batchs=np.array_split(label_all,split_size)
    
    save_dir="{}/epoch_{:03d}".format(model_dir,e)
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
    loss_epoch=[]
    acc_epoch=[]
    batch_number=0
    for data,label in zip(data_batchs,label_batchs):
        fd={net.X:data, net.Y: label}
#         train_step=net.train_step(fd)
#         net.save_hidden_state(save_dir,fd)
        loss_epoch.append(net.batch_loss.eval(fd))
#         acc_epoch.append(net.mean_accuracy.eval(fd))
        print("epoch_{:03d}-batch_{:03d}: loss={}".format(e,batch_number,loss_epoch[-1]))
        batch_number+=1
        
    loss_all.append(loss_epoch)
    acc_all.append(acc_epoch)
    net.save(save_dir,loss_all,acc_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))

init network
training
epoch_000-batch_000: loss=0.6897974826657185
epoch_000-batch_001: loss=0.6906845889811851
epoch_000-batch_002: loss=0.6896974887996004
epoch_000-batch_003: loss=0.6913641449402903
epoch 0 took 265 seconds


SystemExit: 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
